In [12]:

from keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np




In [13]:
def preprocessingImages(path, type):
    """
    Input: Path
    Output: Preprocessed images using ImageDataGenerator
    """
    if type == "train":
        image_data = ImageDataGenerator(
            zoom_range=0.2,
            shear_range=0.2,
            preprocessing_function= preprocess_input,
            horizontal_flip=True,
        )
    else:
        # Only preprocess validation/testing data
        image_data = ImageDataGenerator(
            preprocessing_function= preprocess_input
        )
    
    # Ensure images are resized to 224x224
    image = image_data.flow_from_directory(
        directory=path,
        target_size=(224, 224),  # Matches model input size
        batch_size=32,
        class_mode="binary"  # Must match the type of classification
    )
    
    return image


In [14]:
test_data = preprocessingImages("split_dataset/test", "test")

Found 114 images belonging to 2 classes.


In [15]:
# Load the best fit model

model = load_model("bestmodel.keras")

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_data)

print(f"Our model accuracy is {accuracy*100:.2f} %")
print(f"Our model loss is {loss:.4f}")  

4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 225ms/step - accuracy: 0.9850 - loss: 0.0486
Our model accuracy is 99.12 %
Our model loss is 0.0288


In [16]:
def get_prediction(path):
    img = image.load_img(path, target_size=(224, 224))

    input_arr = img_to_array(img)
    input_arr = np.expand_dims(input_arr, axis=0)
    input_arr = input_arr / 255.0

    # Predict with the model
    prob = model.predict(input_arr)[0][0]  # Extract scalar float
    # If prob >= 0.5 => class 1 (tumor), else class 0 (healthy)
    pred = 1 if prob >= 0.5 else 0
    return pred, prob  # Return both prediction and probability

def predict_img(path):

    # Predict with the model
    pred = get_prediction(path)

    if pred == 0:
        print("The MRI image is of a HEALTHY BRAIN")
    else:
        print("The MRI image indicates a TUMOR")
        
        # Display the input image
    plt.imshow(input_arr[0])
    plt.title("input image")
    plt.show()


In [17]:
import os
from tensorflow.keras.preprocessing import image


def evaluate_folder(folder, type):
    image_paths = [
        os.path.join(folder, f) 
        for f in os.listdir(folder)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ]
    h = 1
    type_txt = "TUMOR"
    if type == "no":
        type_txt = "HEALTHY"
        h = 0
    total = 0
    correct = 0
    misclassified = []

    for img_path in image_paths:
        pred, prob = get_prediction(img_path)  # Get prediction and probability

        total += 1
        if pred == h:
            correct += 1
        else:
            misclassified.append((img_path, prob))  # Append misclassified with probability

    success_percentage = (correct / total) * 100
    print(f"\nTotal images in '{type}' folder: {total}")
    print(f"Correctly predicted as {type_txt}: {correct}")
    print(f"Success percentage: {success_percentage:.2f}%")

    if misclassified:
        print(f"\nMisclassified {type_txt} images:")
        for img_path, prob in misclassified:
            print(f"Image: {img_path}, Probability: {prob:.4f}")

    return success_percentage

In [18]:
# Go through the yes folder and check

yes_folder = "brain-tumor-mri-dataset/Brain_Tumor_Detection/yes"
evaluate_folder(yes_folder, "yes")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 673ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━

96.62576687116564

In [19]:
# Go through the no folder and check

no_folder = "brain-tumor-mri-dataset/Brain_Tumor_Detection/no"
evaluate_folder(no_folder, "no")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━

98.7991266375546

In [20]:
# Download new data from the internet

import requests
import zipfile
import shutil

# Step 0: Define the URL and file paths
url = "https://www.kaggle.com/api/v1/datasets/download/shadyeldakrory/shadyfinal3"

# Get the current directory
current_directory = os.getcwd()

# Define the output file path and extraction folder
output_file = os.path.join(current_directory, "brain-tumor-dataset.zip")
extract_to = os.path.join(current_directory, "brain-tumor-dataset")
dataset_check = os.path.join(current_directory, "dataset_check")  # Folder for `yes` and `no`
yes_folder = os.path.join(dataset_check, "yes")
no_folder = os.path.join(dataset_check, "no")

# Function to check if folders exist and contain images
def folder_has_images(folder_path):
    if os.path.exists(folder_path) and os.listdir(folder_path):
        # Check if at least one file is an image (e.g., jpg, png)
        for file in os.listdir(folder_path):
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                return True
    return False

# Helper function to handle duplicate files
def move_file_with_rename(src, dest_folder):
    file_name = os.path.basename(src)
    dest_path = os.path.join(dest_folder, file_name)
    counter = 1
    while os.path.exists(dest_path):
        # Add a counter to the filename to avoid overwriting
        file_name, file_extension = os.path.splitext(os.path.basename(src))
        dest_path = os.path.join(dest_folder, f"{file_name}_{counter}{file_extension}")
        counter += 1
    shutil.move(src, dest_path)

# Check if dataset_check folder already exists and is organized
if folder_has_images(yes_folder) and folder_has_images(no_folder):
    print(f"Dataset already organized in {dataset_check}. Skipping download and extraction.")
else:
    # Create `dataset_check` and its subfolders if they don't exist
    os.makedirs(yes_folder, exist_ok=True)
    os.makedirs(no_folder, exist_ok=True)

    # Step 1: Download the file
    print("Downloading the dataset...")
    response = requests.get(url, allow_redirects=True)
    with open(output_file, "wb") as file:
        file.write(response.content)
    print(f"Dataset downloaded to {output_file}")

    # Step 2: Unzip the file
    if zipfile.is_zipfile(output_file):
        print("Extracting the dataset...")
        with zipfile.ZipFile(output_file, "r") as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Dataset extracted to {extract_to}")

        # Remove the ZIP file after extraction
        os.remove(output_file)
        print(f"Deleted the ZIP file: {output_file}")
    else:
        print("The downloaded file is not a valid zip file.")
        exit()

    # Step 3: Organize files into `yes` and `no` folders inside `dataset_check`
    print("Organizing files into 'yes' and 'no' folders inside 'dataset_check'...")

    # Define the folder names that contain tumor and no tumor files
    tumor_classes = ["Glioma", "Meningioma", "Pituitary"]
    no_tumor_class = "no_tumor"

    # Iterate through the extracted dataset folders (test, train, validation)
    for folder_name in ["test", "train", "validation"]:
        folder_path = os.path.join(extract_to, folder_name)
        if not os.path.exists(folder_path):
            print(f"Folder {folder_path} does not exist, skipping...")
            continue

        # Process each subfolder inside the dataset (e.g., Glioma, no_tumor, etc.)
        for subfolder in os.listdir(folder_path):
            subfolder_path = os.path.join(folder_path, subfolder)

            if subfolder in tumor_classes:
                # Move all tumor files to the 'yes' folder
                for file_name in os.listdir(subfolder_path):
                    file_path = os.path.join(subfolder_path, file_name)
                    move_file_with_rename(file_path, yes_folder)
            elif subfolder == no_tumor_class:
                # Move all no tumor files to the 'no' folder
                for file_name in os.listdir(subfolder_path):
                    file_path = os.path.join(subfolder_path, file_name)
                    move_file_with_rename(file_path, no_folder)

    # Step 4: Cleanup empty folders
    shutil.rmtree(extract_to)
    print(f"Cleaned up extracted dataset folder: {extract_to}")

    print("Files organized successfully!")
    print(f"Tumor files are in: {yes_folder}")
    print(f"No tumor files are in: {no_folder}")


Dataset already organized in /Users/nestorxx-xx/Library/Mobile Documents/com~apple~CloudDocs/CODE/Visual Studio/MRI_Tumor_Detection/Deep_Learning_Model/dataset_check. Skipping download and extraction.


In [21]:
yes_folder = "dataset_check/yes"
evaluate_folder(yes_folder, "yes")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━

92.2084676999896

In [22]:
no_folder = "dataset_check/no"
evaluate_folder(no_folder, "no")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━

95.80209895052474